In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import torch
import torch.nn as nn
import numpy as np
from faker import Faker

In [6]:
# Cargar los datos originales
semilla = pd.read_excel('tabla_sintetica_muestra_400.xlsx')

# Eliminar columnas innecesarias
semilla = semilla.drop(columns=['Unnamed: 0'])

# Codificar las variables categóricas
label_encoder_empresa = LabelEncoder()
semilla['Empresa'] = label_encoder_empresa.fit_transform(semilla['Empresa'])

label_encoder_trimestre = LabelEncoder()
semilla['Trimestre'] = label_encoder_trimestre.fit_transform(semilla['Trimestre'])

# Excluir 'Año', 'Inversión Inicial', 'Tipo de descuento' de las columnas numéricas para normalización
numerical_columns_without_year = ['Flujo de Caja (USD)', 'Ingresos Anuales (USD)', 'Costos Anuales (USD)',
                                  'Competidores Actuales', 'Proyección Demanda (Unidades)']

# Normalizar las columnas numéricas sin 'Año'
scaler_without_year = MinMaxScaler()
semilla[numerical_columns_without_year] = scaler_without_year.fit_transform(semilla[numerical_columns_without_year])

# Verificar el DataFrame preprocesado
print(semilla.head())

    Año  Inversión Inicial (USD)  Flujo de Caja (USD)  Tipo de descuento (%)  \
0  2019                849507.03             1.000000                   6.89   
1  2019                849507.03             1.000000                   6.89   
2  2019                849507.03             1.000000                   6.89   
3  2019                849507.03             1.000000                   6.89   
4  2020                849507.03             0.508984                   6.89   

   Ingresos Anuales (USD)  Costos Anuales (USD)  Competidores Actuales  \
0                0.057018              0.010583               0.666667   
1                0.057018              0.010583               0.666667   
2                0.057018              0.010583               0.666667   
3                0.057018              0.010583               0.666667   
4                0.976043              0.978423               0.833333   

   Proyección Demanda (Unidades)  Empresa  Trimestre  
0                  

In [7]:
# Definir el modelo CVAE
class CVAE(nn.Module):
    def __init__(self, input_dim, latent_dim, condition_dim):
        super(CVAE, self).__init__()
        
        # Encoder: toma la entrada y las condiciones
        self.fc1 = nn.Linear(input_dim + condition_dim, 128)
        self.fc2_mean = nn.Linear(128, latent_dim)
        self.fc2_log_var = nn.Linear(128, latent_dim)
        
        # Decoder: toma las condiciones y el espacio latente
        self.fc3 = nn.Linear(latent_dim + condition_dim, 128)
        self.fc4 = nn.Linear(128, input_dim)
    
    def encoder(self, x, c):
        h = torch.relu(self.fc1(torch.cat([x, c], dim=1)))
        z_mean = self.fc2_mean(h)
        z_log_var = self.fc2_log_var(h)
        return z_mean, z_log_var
    
    def reparameterize(self, z_mean, z_log_var):
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std
    
    def decoder(self, z, c):
        h = torch.relu(self.fc3(torch.cat([z, c], dim=1)))
        return torch.sigmoid(self.fc4(h))
    
    def forward(self, x, c):
        z_mean, z_log_var = self.encoder(x, c)
        z = self.reparameterize(z_mean, z_log_var)
        return self.decoder(z, c), z_mean, z_log_var

# Parámetros del CVAE
input_dim = 2  # Aquí puedes ajustar el número de columnas que dependen de la empresa (ej. Inversión Inicial y Tipo de Descuento)
latent_dim = 2  # Dimensiones del espacio latente
condition_dim = 3  # Condiciones: Empresa, Año, Trimestre

# Instanciar el modelo CVAE
cvae = CVAE(input_dim=input_dim, latent_dim=latent_dim, condition_dim=condition_dim)

# Número de muestras a generar
num_samples = 5000

# Generar latentes y condiciones para el CVAE
z_cvae = torch.randn(num_samples, latent_dim)
empresa_condiciones = torch.randint(0, len(label_encoder_empresa.classes_), (num_samples, 1)).float()
anio_condiciones = torch.randint(2019, 2024, (num_samples, 1)).float()
trimestre_condiciones = torch.randint(1, 5, (num_samples, 1)).float()

# Concatenar las condiciones de empresa, año y trimestre
condiciones_completas = torch.cat([empresa_condiciones, anio_condiciones, trimestre_condiciones], dim=1)

# Generar datos categóricos (incluyendo inversión inicial y tipo de descuento) con el CVAE
with torch.no_grad():
    generated_data_categoricas = cvae.decoder(z_cvae, condiciones_completas)

# Tomar las columnas categóricas (empresa, año, trimestre) y las dependientes (inversión inicial y tipo de descuento)
generated_empresa = label_encoder_empresa.inverse_transform(empresa_condiciones.int().squeeze())
generated_anio = anio_condiciones.int().squeeze()
generated_trimestre = trimestre_condiciones.int().squeeze()

generated_inversion_inicial = generated_data_categoricas[:, 0].numpy()  # Primer columna: Inversión Inicial
generated_tipo_descuento = generated_data_categoricas[:, 1].numpy()  # Segunda columna: Tipo de Descuento

In [8]:
# Definir el VAE
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2_mean = nn.Linear(128, latent_dim)
        self.fc2_log_var = nn.Linear(128, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 128)
        self.fc4 = nn.Linear(128, input_dim)
    
    def encoder(self, x):
        h = torch.relu(self.fc1(x))
        z_mean = self.fc2_mean(h)
        z_log_var = self.fc2_log_var(h)
        return z_mean, z_log_var
    
    def reparameterize(self, z_mean, z_log_var):
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std
    
    def decoder(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))
    
    def forward(self, x):
        z_mean, z_log_var = self.encoder(x)
        z = self.reparameterize(z_mean, z_log_var)
        return self.decoder(z), z_mean, z_log_var

# Parámetros del VAE
input_dim = len(numerical_columns_without_year)  # Número de columnas numéricas restantes
latent_dim = 2

# Instanciar el modelo VAE
vae = VAE(input_dim=input_dim, latent_dim=latent_dim)

# Generar datos numéricos con el VAE
z_vae = torch.randn(num_samples, latent_dim)
with torch.no_grad():
    generated_data_numericas = vae.decoder(z_vae)

# Desnormalizar los datos numéricos
generated_data_numericas = scaler_without_year.inverse_transform(generated_data_numericas.numpy())

# Dividir los datos numéricos generados
generated_ingresos_anuales = generated_data_numericas[:, 0]
generated_costos_anuales = generated_data_numericas[:, 1]
generated_competidores = generated_data_numericas[:, 2]
generated_proyeccion_demanda = generated_data_numericas[:, 3]

# Calcular el Flujo de Caja como Ingresos Anuales - Costos Anuales
generated_flujo_caja = generated_ingresos_anuales - generated_costos_anuales

In [11]:
# Crear DataFrame final con todas las columnas generadas
generated_df = pd.DataFrame({
    'Empresa': generated_empresa,
    'Año': generated_anio,
    'Trimestre': generated_trimestre,
    'Inversión Inicial (USD)': generated_inversion_inicial,
    'Tipo de descuento (%)': generated_tipo_descuento,
    'Flujo de Caja (USD)': generated_flujo_caja,
    'Ingresos Anuales (USD)': generated_ingresos_anuales,
    'Costos Anuales (USD)': generated_costos_anuales,
    'Competidores Actuales': generated_competidores,
    'Proyección Demanda (Unidades)': generated_proyeccion_demanda
})

# Verificar que los datos de las columnas categóricas estén bien distribuidos
print(generated_df.head())

# Verificar que para una misma empresa en un mismo año se mantengan las columnas "Inversión Inicial" y "Tipo de descuento"
for empresa in np.unique(generated_df['Empresa']):
    empresa_df = generated_df[generated_df['Empresa'] == empresa]
    print(f"\nDatos para la empresa {empresa}:")
    print(empresa_df[['Año', 'Inversión Inicial (USD)', 'Tipo de descuento (%)']].drop_duplicates())

# Guardar los datos generados en un archivo CSV
generated_df.to_csv('datos_sinteticos_cvae_vae.csv', index=False)

print("Datos generados guardados exitosamente en 'datos_sinteticos_cvae_vae.csv'.")

          Empresa   Año  Trimestre  Inversión Inicial (USD)  \
0        Creality  2019          3                      0.0   
1        Anycubic  2022          3                      0.0   
2      FlashForge  2022          3                      0.0   
3     Aleph Farms  2022          1                      0.0   
4  Prusa Research  2023          3                      0.0   

   Tipo de descuento (%)  Flujo de Caja (USD)  Ingresos Anuales (USD)  \
0                    1.0         -327377824.0              14422393.0   
1                    1.0         -348494432.0               9941992.0   
2                    1.0         -329777440.0              22580342.0   
3                    1.0         -351908672.0               6581649.5   
4                    1.0         -363366656.0              -6897249.5   

   Costos Anuales (USD)  Competidores Actuales  Proyección Demanda (Unidades)  
0           341800224.0            302070976.0                       6.532720  
1           358436416.

In [12]:
datos_sinteticos = pd.read_csv('datos_sinteticos_cvae_vae.csv')

In [13]:
# Modificar Inversión Inicial según los rangos dados
np.random.seed(42)
datos_sinteticos['Inversión Inicial (USD)'] = np.random.normal(
    loc=6000000, scale=5000000, size=len(datos_sinteticos)
).clip(850000, 15000000)  # Limitar los valores dentro del rango

In [14]:
# Modificar Tipo de Descuento según los rangos dados
datos_sinteticos['Tipo de descuento (%)'] = np.random.normal(
    loc=7, scale=2.5, size=len(datos_sinteticos)
).clip(3, 15)  # Limitar los valores dentro del rango

In [15]:
# Generar Ingresos Anuales y Costos Anuales con la condición que 75% de las veces Ingresos > Costos
num_samples = len(datos_sinteticos)
num_greater_cases = int(0.75 * num_samples)

# Generar los ingresos para 75% de los casos donde Ingresos > Costos
ingresos_mayor = np.random.normal(loc=80e6, scale=30e6, size=num_greater_cases).clip(90e6, 123e6)
costos_menor = np.random.normal(loc=350e6, scale=200e6, size=num_greater_cases).clip(50e6, 627e6)

# Generar los ingresos y costos para el 25% de los casos donde Ingresos < Costos
ingresos_menor = np.random.normal(loc=80e6, scale=30e6, size=num_samples - num_greater_cases).clip(90e6, 123e6)
costos_mayor = np.random.normal(loc=350e6, scale=200e6, size=num_samples - num_greater_cases).clip(50e6, 627e6)

# Concatenar ambos
ingresos_anuales = np.concatenate([ingresos_mayor, ingresos_menor])
costos_anuales = np.concatenate([costos_menor, costos_mayor])

# Mezclar aleatoriamente los datos
np.random.shuffle(ingresos_anuales)
np.random.shuffle(costos_anuales)

# Asignar a las columnas
datos_sinteticos['Ingresos Anuales (USD)'] = ingresos_anuales
datos_sinteticos['Costos Anuales (USD)'] = costos_anuales

In [16]:
# Calcular el flujo de caja como Ingresos Anuales - Costos Anuales
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

In [17]:
# Modificar Competidores Actuales
datos_sinteticos['Competidores Actuales'] = np.random.randint(5, 16, size=len(datos_sinteticos))

# Modificar Proyección de Demanda
datos_sinteticos['Proyección Demanda (Unidades)'] = np.random.randint(600, 2001, size=len(datos_sinteticos))

In [18]:
# Revisar las combinaciones de Empresa, Año y Trimestre
for empresa in datos_sinteticos['Empresa'].unique():
    empresa_df = datos_sinteticos[datos_sinteticos['Empresa'] == empresa]
    for año in range(2019, 2024):
        year_quarters = empresa_df[empresa_df['Año'] == año]['Trimestre'].values
        if len(year_quarters) != 4 or sorted(year_quarters) != [1, 2, 3, 4]:
            print(f"La empresa {empresa} no tiene 4 trimestres correctos para el año {año}. Verificar.")

La empresa Creality no tiene 4 trimestres correctos para el año 2019. Verificar.
La empresa Creality no tiene 4 trimestres correctos para el año 2020. Verificar.
La empresa Creality no tiene 4 trimestres correctos para el año 2021. Verificar.
La empresa Creality no tiene 4 trimestres correctos para el año 2022. Verificar.
La empresa Creality no tiene 4 trimestres correctos para el año 2023. Verificar.
La empresa Anycubic no tiene 4 trimestres correctos para el año 2019. Verificar.
La empresa Anycubic no tiene 4 trimestres correctos para el año 2020. Verificar.
La empresa Anycubic no tiene 4 trimestres correctos para el año 2021. Verificar.
La empresa Anycubic no tiene 4 trimestres correctos para el año 2022. Verificar.
La empresa Anycubic no tiene 4 trimestres correctos para el año 2023. Verificar.
La empresa FlashForge no tiene 4 trimestres correctos para el año 2019. Verificar.
La empresa FlashForge no tiene 4 trimestres correctos para el año 2020. Verificar.
La empresa FlashForge no

In [21]:
# Verificar cuántos trimestres tiene Creality en 2019
creality_2019 = datos_sinteticos[(datos_sinteticos['Empresa'] == 'Creality') & (datos_sinteticos['Año'] == 2019)]
print(creality_2019[['Trimestre']])

      Trimestre
0             3
7             4
39            4
129           4
179           3
342           3
415           1
621           2
624           2
636           3
647           4
668           3
852           3
901           4
1117          1
1139          2
1172          4
1540          3
1541          4
1631          3
1676          4
1715          4
1878          3
1895          1
1905          3
2028          4
2103          4
2194          4
2249          4
2359          1
2505          3
2555          1
2656          3
2982          2
3018          1
3107          4
3246          2
3327          4
3335          3
3396          1
3422          4
3466          3
3486          1
3509          3
3715          4
3976          3
4109          3
4115          4
4167          1
4262          1
4307          2
4498          1
4503          3
4522          3
4555          2
4587          2
4621          1


In [39]:
# Extraer los nombres únicos de empresas existentes
empresas_existentes = datos_sinteticos['Empresa'].unique().tolist()

# Crear una instancia de Faker para generar nombres de empresas realistas
fake = Faker()

# Número total de empresas que necesitamos
num_total_empresas = 250

# Número de nuevas empresas que necesitamos generar para llegar a 250
num_nuevas_empresas = num_total_empresas - len(empresas_existentes)

# Generar nombres de nuevas empresas (que no estén ya en la lista existente)
nuevas_empresas = []
while len(nuevas_empresas) < num_nuevas_empresas:
    nuevo_nombre = fake.company()
    if nuevo_nombre not in empresas_existentes and nuevo_nombre not in nuevas_empresas:
        nuevas_empresas.append(nuevo_nombre)

# Unir las empresas existentes con las nuevas, asegurando que las existentes queden al principio
lista_empresas_final = empresas_existentes + nuevas_empresas

# Verificar que tenemos exactamente 250 empresas
assert len(lista_empresas_final) == 250, "La lista de empresas debe contener exactamente 250 nombres."

# Generar las combinaciones de años y trimestres para cada empresa
años = list(range(2019, 2024))  # Años de 2019 a 2023
trimestres = [1, 2, 3, 4]  # Cuatro trimestres por año

# Cada empresa debe tener 5 años * 4 trimestres = 20 registros
num_registros_empresa = len(años) * len(trimestres)  # 20 registros por empresa

# Verificar que 250 empresas con 20 registros cada una ocupan los 5000 registros existentes
assert num_total_empresas * num_registros_empresa == len(datos_sinteticos), "Los registros totales deben coincidir con los registros del dataset."

# Crear una lista repetida de empresas
empresas_repetidas = np.repeat(lista_empresas_final, num_registros_empresa)

# Crear una lista repetida de años y trimestres para todas las empresas
años_repetidos = np.tile(np.repeat(años, len(trimestres)), num_total_empresas)
trimestres_repetidos = np.tile(trimestres, len(años) * num_total_empresas)

# Verificar las longitudes
assert len(empresas_repetidas) == len(datos_sinteticos), f"Longitud de empresas: {len(empresas_repetidas)}, registros del dataset: {len(datos_sinteticos)}"
assert len(años_repetidos) == len(datos_sinteticos), f"Longitud de años: {len(años_repetidos)}, registros del dataset: {len(datos_sinteticos)}"
assert len(trimestres_repetidos) == len(datos_sinteticos), f"Longitud de trimestres: {len(trimestres_repetidos)}, registros del dataset: {len(datos_sinteticos)}"

# Asignar estas listas al dataframe
datos_sinteticos['Empresa'] = empresas_repetidas
datos_sinteticos['Año'] = años_repetidos
datos_sinteticos['Trimestre'] = trimestres_repetidos

# Verificar la distribución de empresas, años y trimestres
print(datos_sinteticos[['Empresa', 'Año', 'Trimestre']].head(20))

# Guardar el archivo actualizado
datos_sinteticos.to_csv('datos_sinteticos_ajustados.csv', index=False)

     Empresa   Año  Trimestre
0   Creality  2019          1
1   Creality  2019          2
2   Creality  2019          3
3   Creality  2019          4
4   Creality  2020          1
5   Creality  2020          2
6   Creality  2020          3
7   Creality  2020          4
8   Creality  2021          1
9   Creality  2021          2
10  Creality  2021          3
11  Creality  2021          4
12  Creality  2022          1
13  Creality  2022          2
14  Creality  2022          3
15  Creality  2022          4
16  Creality  2023          1
17  Creality  2023          2
18  Creality  2023          3
19  Creality  2023          4


In [40]:
# Normalización de Inversión Inicial (USD)
datos_sinteticos['Inversión Inicial (USD)'] = np.random.normal(loc=6000000, scale=5000000, size=len(datos_sinteticos))
datos_sinteticos['Inversión Inicial (USD)'] = np.clip(datos_sinteticos['Inversión Inicial (USD)'], 850000, 15000000)

# Normalización de Tipo de Descuento (%)
datos_sinteticos['Tipo de descuento (%)'] = np.random.normal(loc=7, scale=2.5, size=len(datos_sinteticos))
datos_sinteticos['Tipo de descuento (%)'] = np.clip(datos_sinteticos['Tipo de descuento (%)'], 3, 15)

# Normalización de Ingresos Anuales (USD)
datos_sinteticos['Ingresos Anuales (USD)'] = np.random.normal(loc=80e6, scale=30e6, size=len(datos_sinteticos))
datos_sinteticos['Ingresos Anuales (USD)'] = np.clip(datos_sinteticos['Ingresos Anuales (USD)'], 90e6, 123e6)

# Normalización de Costos Anuales (USD)
datos_sinteticos['Costos Anuales (USD)'] = np.random.normal(loc=350e6, scale=200e6, size=len(datos_sinteticos))
datos_sinteticos['Costos Anuales (USD)'] = np.clip(datos_sinteticos['Costos Anuales (USD)'], 50e6, 627e6)

# Asegurar que el 75% de los registros tengan Ingresos > Costos
mask = np.random.rand(len(datos_sinteticos)) < 0.75
datos_sinteticos.loc[mask, 'Ingresos Anuales (USD)'] = np.maximum(datos_sinteticos.loc[mask, 'Ingresos Anuales (USD)'], 
                                                                  datos_sinteticos.loc[mask, 'Costos Anuales (USD)'])

# Competidores Actuales
datos_sinteticos['Competidores Actuales'] = np.random.randint(5, 16, size=len(datos_sinteticos))

# Proyección de Demanda (Unidades)
datos_sinteticos['Proyección Demanda (Unidades)'] = np.random.randint(600, 2001, size=len(datos_sinteticos))

# Flujo de Caja (USD) = Ingresos Anuales - Costos Anuales
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

In [41]:
# Verificar que cada empresa tenga 4 trimestres por año
for empresa in lista_empresas_final:
    for año in años:
        trimestres = datos_sinteticos[(datos_sinteticos['Empresa'] == empresa) & (datos_sinteticos['Año'] == año)]['Trimestre']
        assert len(trimestres) == 4, f"La empresa {empresa} no tiene 4 trimestres en el año {año}."

# Verificar la distribución de Inversión Inicial (USD)
print(datos_sinteticos['Inversión Inicial (USD)'].describe())

# Verificar la distribución de Tipo de Descuento (%)
print(datos_sinteticos['Tipo de descuento (%)'].describe())

count    5.000000e+03
mean     6.277258e+06
std      4.181165e+06
min      8.500000e+05
25%      2.516901e+06
50%      5.890421e+06
75%      9.296441e+06
max      1.500000e+07
Name: Inversión Inicial (USD), dtype: float64
count    5000.000000
mean        7.075459
std         2.381457
min         3.000000
25%         5.267346
50%         7.037094
75%         8.752622
max        15.000000
Name: Tipo de descuento (%), dtype: float64


In [42]:
# Exportar el dataset para comprobar resultados
datos_sinteticos.to_csv('datos_sinteticos_final_con_nuevas_empresas.csv', index=False)

In [43]:
# Definir las columnas que necesitan ajuste de decimales
columnas_numericas = ['Inversión Inicial (USD)', 'Flujo de Caja (USD)', 'Tipo de descuento (%)',
                      'Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Competidores Actuales',
                      'Proyección Demanda (Unidades)']

# 1. Asegurar que Ingresos Anuales sean mayores que Costos Anuales en el 75% de los casos
total_registros = len(datos_sinteticos)
porcentaje_mayor_ingresos = 0.75

# Identificar índices donde hacer este ajuste
indices_mayores = np.random.choice(datos_sinteticos.index, size=int(total_registros * porcentaje_mayor_ingresos), replace=False)
indices_menores = np.setdiff1d(datos_sinteticos.index, indices_mayores)

# Ajustar los registros donde los ingresos deben ser mayores
datos_sinteticos.loc[indices_mayores, 'Ingresos Anuales (USD)'] = np.maximum(
    datos_sinteticos.loc[indices_mayores, 'Ingresos Anuales (USD)'],
    datos_sinteticos.loc[indices_mayores, 'Costos Anuales (USD)'] + 1000  # Asegurarse de que los ingresos sean mayores
)

# Ajustar los registros donde los costos son mayores (o iguales)
datos_sinteticos.loc[indices_menores, 'Costos Anuales (USD)'] = np.maximum(
    datos_sinteticos.loc[indices_menores, 'Costos Anuales (USD)'],
    datos_sinteticos.loc[indices_menores, 'Ingresos Anuales (USD)'] + 1000  # Asegurarse de que los costos sean mayores
)

# Asegurarse de que no existan ingresos iguales a costos
iguales_indices = datos_sinteticos[datos_sinteticos['Ingresos Anuales (USD)'] == datos_sinteticos['Costos Anuales (USD)']].index
datos_sinteticos.loc[iguales_indices, 'Ingresos Anuales (USD)'] += 1000  # Desempatar si hay igualdad

# 2. Limitar los decimales a dos en todas las columnas numéricas
for columna in columnas_numericas:
    datos_sinteticos[columna] = datos_sinteticos[columna].round(2)

# 3. Guardar el dataset ajustado
datos_sinteticos.to_csv('datos_sinteticos_afinados.csv', index=False)

# Verificar los ajustes
print(datos_sinteticos[['Ingresos Anuales (USD)', 'Costos Anuales (USD)']].head(10))

   Ingresos Anuales (USD)  Costos Anuales (USD)
0            1.260114e+08          1.260104e+08
1            1.260816e+08          1.260806e+08
2            3.279909e+08          3.279899e+08
3            4.753582e+08          4.753572e+08
4            9.000000e+07          4.642224e+08
5            5.911442e+08          5.911452e+08
6            1.986249e+08          1.986239e+08
7            2.760352e+08          2.760342e+08
8            4.770941e+08          4.770931e+08
9            4.941551e+08          4.941541e+08


In [44]:
datos_sinteticos

,Empresa,Año,Trimestre,Inversión Inicial (USD),Tipo de descuento (%),Flujo de Caja (USD),Ingresos Anuales (USD),Costos Anuales (USD),Competidores Actuales,Proyección Demanda (Unidades)
0,Creality,2019,1,5848413.64,11.21,-7.184484e+06,1.260114e+08,1.260104e+08,10,1229
1,Creality,2019,2,12726821.65,3.68,0.000000e+00,1.260816e+08,1.260806e+08,6,1453
2,Creality,2019,3,13691361.35,5.78,0.000000e+00,3.279909e+08,3.279899e+08,15,995
3,Creality,2019,4,4231370.70,6.73,0.000000e+00,4.753582e+08,4.753572e+08,6,1522
4,Creality,2020,1,5124946.55,9.06,-3.742224e+08,9.000000e+07,4.642224e+08,10,1514
...,...,...,...,...,...,...,...,...,...,...
4995,Allen-Black,2022,4,9801133.75,6.68,-2.353518e+07,1.170680e+08,1.406032e+08,10,713
4996,Allen-Black,2023,1,10227212.19,3.06,4.000000e+07,9.000000e+07,5.000000e+07,9,1853
4997,Allen-Black,2023,2,12298146.14,6.05,0.000000e+00,5.265773e+08,5.265763e+08,13,1689
4998,Allen-Black,2023,3,3876207.22,6.70,-6.458149e+06,9.000000e+07,9.645815e+07,9,1154


In [46]:
# Definir las columnas que necesitan ajuste de decimales
columnas_numericas = ['Inversión Inicial (USD)', 'Flujo de Caja (USD)', 'Tipo de descuento (%)',
                      'Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Competidores Actuales',
                      'Proyección Demanda (Unidades)']

# 1. Asegurar que Ingresos Anuales sean mayores que Costos Anuales en el 75% de los casos
total_registros = len(datos_sinteticos)
porcentaje_mayor_ingresos = 0.75

# Identificar índices donde hacer este ajuste
indices_mayores = np.random.choice(datos_sinteticos.index, size=int(total_registros * porcentaje_mayor_ingresos), replace=False)
indices_menores = np.setdiff1d(datos_sinteticos.index, indices_mayores)

# Ajustar los registros donde los ingresos deben ser mayores
datos_sinteticos.loc[indices_mayores, 'Ingresos Anuales (USD)'] = np.maximum(
    datos_sinteticos.loc[indices_mayores, 'Ingresos Anuales (USD)'],
    datos_sinteticos.loc[indices_mayores, 'Costos Anuales (USD)'] + np.random.uniform(5000, 15000, size=len(indices_mayores))
)

# Ajustar los registros donde los costos son mayores (o iguales)
datos_sinteticos.loc[indices_menores, 'Costos Anuales (USD)'] = np.maximum(
    datos_sinteticos.loc[indices_menores, 'Costos Anuales (USD)'],
    datos_sinteticos.loc[indices_menores, 'Ingresos Anuales (USD)'] + np.random.uniform(5000, 15000, size=len(indices_menores))
)

# 2. Verificar si aún hay registros donde los ingresos y costos son iguales
iguales_indices = datos_sinteticos[datos_sinteticos['Ingresos Anuales (USD)'] == datos_sinteticos['Costos Anuales (USD)']].index

# Corregir los registros donde Ingresos Anuales son iguales a Costos Anuales
for idx in iguales_indices:
    if np.random.rand() > 0.5:
        datos_sinteticos.loc[idx, 'Ingresos Anuales (USD)'] += np.random.uniform(1000, 5000)
    else:
        datos_sinteticos.loc[idx, 'Costos Anuales (USD)'] += np.random.uniform(1000, 5000)

# 3. Limitar los decimales a dos en todas las columnas numéricas
for columna in columnas_numericas:
    datos_sinteticos[columna] = datos_sinteticos[columna].round(2)

# 4. Actualizar la columna de Flujo de Caja como la diferencia entre Ingresos y Costos
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

# 5. Guardar el dataset ajustado
datos_sinteticos.to_csv('datos_sinteticos_afinados.csv', index=False)

# Verificar los ajustes
print(datos_sinteticos[['Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Flujo de Caja (USD)']].head(10))

   Ingresos Anuales (USD)  Costos Anuales (USD)  Flujo de Caja (USD)
0            1.260214e+08          1.260104e+08             11050.96
1            1.260816e+08          1.260905e+08             -8904.64
2            3.280041e+08          3.279899e+08             14264.40
3            4.753638e+08          4.753572e+08              6594.86
4            4.642285e+08          4.642224e+08              6085.85
5            5.911545e+08          5.911452e+08              9357.94
6            1.986344e+08          1.986239e+08             10501.64
7            2.760352e+08          2.760465e+08            -11267.17
8            4.771073e+08          4.770931e+08             14201.51
9            4.941611e+08          4.941541e+08              7052.56


In [47]:
datos_sinteticos

,Empresa,Año,Trimestre,Inversión Inicial (USD),Tipo de descuento (%),Flujo de Caja (USD),Ingresos Anuales (USD),Costos Anuales (USD),Competidores Actuales,Proyección Demanda (Unidades)
0,Creality,2019,1,5848413.64,11.21,11050.96,1.260214e+08,1.260104e+08,10,1229
1,Creality,2019,2,12726821.65,3.68,-8904.64,1.260816e+08,1.260905e+08,6,1453
2,Creality,2019,3,13691361.35,5.78,14264.40,3.280041e+08,3.279899e+08,15,995
3,Creality,2019,4,4231370.70,6.73,6594.86,4.753638e+08,4.753572e+08,6,1522
4,Creality,2020,1,5124946.55,9.06,6085.85,4.642285e+08,4.642224e+08,10,1514
...,...,...,...,...,...,...,...,...,...,...
4995,Allen-Black,2022,4,9801133.75,6.68,7492.63,1.406107e+08,1.406032e+08,10,713
4996,Allen-Black,2023,1,10227212.19,3.06,-5826.24,9.000000e+07,9.000583e+07,9,1853
4997,Allen-Black,2023,2,12298146.14,6.05,7410.64,5.265837e+08,5.265763e+08,13,1689
4998,Allen-Black,2023,3,3876207.22,6.70,12377.81,9.647053e+07,9.645815e+07,9,1154


In [48]:
# Definir las columnas numéricas
columnas_numericas = ['Inversión Inicial (USD)', 'Flujo de Caja (USD)', 'Tipo de descuento (%)',
                      'Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Competidores Actuales',
                      'Proyección Demanda (Unidades)']

# 1. Reducir aleatoriamente los Costos Anuales en un porcentaje significativo en la mayoría de los casos
# En este caso, reducimos los costos en un 90% de los registros
porcentaje_reducir = 0.90
total_registros = len(datos_sinteticos)
indices_reducir = np.random.choice(datos_sinteticos.index, size=int(total_registros * porcentaje_reducir), replace=False)

# Aplicar una reducción significativa (por ejemplo, entre 10% y 30%)
datos_sinteticos.loc[indices_reducir, 'Costos Anuales (USD)'] *= np.random.uniform(0.7, 0.9, size=len(indices_reducir))

# 2. Recalcular el Flujo de Caja como la diferencia entre Ingresos Anuales y Costos Anuales
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

# 3. Limitar los decimales a dos en todas las columnas numéricas
for columna in columnas_numericas:
    datos_sinteticos[columna] = datos_sinteticos[columna].round(2)

# 4. Guardar el dataset ajustado
datos_sinteticos.to_csv('datos_sinteticos_final.csv', index=False)

# Verificar los primeros registros después del ajuste
print(datos_sinteticos[['Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Flujo de Caja (USD)']].head(10))

   Ingresos Anuales (USD)  Costos Anuales (USD)  Flujo de Caja (USD)
0            1.260214e+08          9.565245e+07         3.036898e+07
1            1.260816e+08          1.031075e+08         2.297410e+07
2            3.280041e+08          2.394594e+08         8.854476e+07
3            4.753638e+08          3.487863e+08         1.265774e+08
4            4.642285e+08          4.148275e+08         4.940093e+07
5            5.911545e+08          5.003177e+08         9.083687e+07
6            1.986344e+08          1.718002e+08         2.683418e+07
7            2.760352e+08          2.169193e+08         5.911594e+07
8            4.771073e+08          3.431273e+08         1.339799e+08
9            4.941611e+08          4.433479e+08         5.081323e+07


In [49]:
datos_sinteticos

,Empresa,Año,Trimestre,Inversión Inicial (USD),Tipo de descuento (%),Flujo de Caja (USD),Ingresos Anuales (USD),Costos Anuales (USD),Competidores Actuales,Proyección Demanda (Unidades)
0,Creality,2019,1,5848413.64,11.21,3.036898e+07,1.260214e+08,9.565245e+07,10,1229
1,Creality,2019,2,12726821.65,3.68,2.297410e+07,1.260816e+08,1.031075e+08,6,1453
2,Creality,2019,3,13691361.35,5.78,8.854476e+07,3.280041e+08,2.394594e+08,15,995
3,Creality,2019,4,4231370.70,6.73,1.265774e+08,4.753638e+08,3.487863e+08,6,1522
4,Creality,2020,1,5124946.55,9.06,4.940093e+07,4.642285e+08,4.148275e+08,10,1514
...,...,...,...,...,...,...,...,...,...,...
4995,Allen-Black,2022,4,9801133.75,6.68,7.492630e+03,1.406107e+08,1.406032e+08,10,713
4996,Allen-Black,2023,1,10227212.19,3.06,1.576700e+07,9.000000e+07,7.423300e+07,9,1853
4997,Allen-Black,2023,2,12298146.14,6.05,1.574466e+08,5.265837e+08,3.691371e+08,13,1689
4998,Allen-Black,2023,3,3876207.22,6.70,2.262618e+07,9.647053e+07,7.384435e+07,9,1154


In [50]:
# Recalcular Flujo de Caja como la diferencia entre Ingresos Anuales y Costos Anuales
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

# Limitar los decimales de la columna Flujo de Caja a dos
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Flujo de Caja (USD)'].round(2)

In [51]:
datos_sinteticos

,Empresa,Año,Trimestre,Inversión Inicial (USD),Tipo de descuento (%),Flujo de Caja (USD),Ingresos Anuales (USD),Costos Anuales (USD),Competidores Actuales,Proyección Demanda (Unidades)
0,Creality,2019,1,5848413.64,11.21,3.036898e+07,1.260214e+08,9.565245e+07,10,1229
1,Creality,2019,2,12726821.65,3.68,2.297410e+07,1.260816e+08,1.031075e+08,6,1453
2,Creality,2019,3,13691361.35,5.78,8.854476e+07,3.280041e+08,2.394594e+08,15,995
3,Creality,2019,4,4231370.70,6.73,1.265774e+08,4.753638e+08,3.487863e+08,6,1522
4,Creality,2020,1,5124946.55,9.06,4.940093e+07,4.642285e+08,4.148275e+08,10,1514
...,...,...,...,...,...,...,...,...,...,...
4995,Allen-Black,2022,4,9801133.75,6.68,7.492630e+03,1.406107e+08,1.406032e+08,10,713
4996,Allen-Black,2023,1,10227212.19,3.06,1.576700e+07,9.000000e+07,7.423300e+07,9,1853
4997,Allen-Black,2023,2,12298146.14,6.05,1.574466e+08,5.265837e+08,3.691371e+08,13,1689
4998,Allen-Black,2023,3,3876207.22,6.70,2.262618e+07,9.647053e+07,7.384435e+07,9,1154


In [52]:
datos_sinteticos.to_csv('datos_sintetizados.csv')

In [53]:
# Eliminar la columna 'Trimestre'
datos_sinteticos = datos_sinteticos.drop(columns=['Trimestre'])

# Agrupar los datos por Empresa y Año, tomando la media de los registros por año
# Esto reducirá los múltiples registros trimestrales a un solo registro anual
datos_sinteticos_anual = datos_sinteticos.groupby(['Empresa', 'Año']).mean().reset_index()

# Limitar los decimales de todas las columnas numéricas a dos decimales
for columna in datos_sinteticos_anual.select_dtypes(include=['float', 'int']).columns:
    datos_sinteticos_anual[columna] = datos_sinteticos_anual[columna].round(2)

# Guardar el nuevo archivo ajustado
datos_sinteticos_anual.to_csv('datos_sinteticos_anual.csv', index=False)

# Verificar los primeros registros después del ajuste
print(datos_sinteticos_anual.head(10))

       Empresa   Año  Inversión Inicial (USD)  Tipo de descuento (%)  \
0   3D Systems  2019               7926786.86                   7.56   
1   3D Systems  2020               9411541.28                   6.37   
2   3D Systems  2021               5734223.12                   5.50   
3   3D Systems  2022               7384371.81                   7.23   
4   3D Systems  2023               6052650.07                   6.79   
5  Aleph Farms  2019               4128589.20                   5.04   
6  Aleph Farms  2020               2006155.28                   6.64   
7  Aleph Farms  2021               8594394.24                   7.70   
8  Aleph Farms  2022               6517581.82                   6.62   
9  Aleph Farms  2023               4855792.85                   7.87   

   Flujo de Caja (USD)  Ingresos Anuales (USD)  Costos Anuales (USD)  \
0          30735102.37            2.878798e+08          2.571447e+08   
1          32758211.96            3.105527e+08          2.77794

In [54]:
datos_sinteticos_anual.to_excel('datos_sinteticos_anual.xlsx')

In [55]:
# Mantener constantes las columnas 'Tipo de descuento' e 'Inversión Inicial' por empresa
tipo_descuento_por_empresa = datos_sinteticos.groupby('Empresa')['Tipo de descuento (%)'].first()
inversion_inicial_por_empresa = datos_sinteticos.groupby('Empresa')['Inversión Inicial (USD)'].first()

# Reasignar las columnas 'Tipo de descuento' e 'Inversión Inicial' por empresa para cada registro
datos_sinteticos['Tipo de descuento (%)'] = datos_sinteticos['Empresa'].map(tipo_descuento_por_empresa)
datos_sinteticos['Inversión Inicial (USD)'] = datos_sinteticos['Empresa'].map(inversion_inicial_por_empresa)

# Recalcular Flujo de Caja como la diferencia entre Ingresos Anuales y Costos Anuales
datos_sinteticos['Flujo de Caja (USD)'] = datos_sinteticos['Ingresos Anuales (USD)'] - datos_sinteticos['Costos Anuales (USD)']

# Guardar el dataset ajustado
datos_sinteticos.to_csv('datos_sinteticos_modificados.csv', index=False)

# Verificar los primeros registros después del ajuste
print(datos_sinteticos[['Empresa', 'Ingresos Anuales (USD)', 'Costos Anuales (USD)', 'Flujo de Caja (USD)', 'Tipo de descuento (%)', 'Inversión Inicial (USD)']].head(10))

    Empresa  Ingresos Anuales (USD)  Costos Anuales (USD)  \
0  Creality            1.260214e+08          9.565245e+07   
1  Creality            1.260816e+08          1.031075e+08   
2  Creality            3.280041e+08          2.394594e+08   
3  Creality            4.753638e+08          3.487863e+08   
4  Creality            4.642285e+08          4.148275e+08   
5  Creality            5.911545e+08          5.003177e+08   
6  Creality            1.986344e+08          1.718002e+08   
7  Creality            2.760352e+08          2.169193e+08   
8  Creality            4.771073e+08          3.431273e+08   
9  Creality            4.941611e+08          4.433479e+08   

   Flujo de Caja (USD)  Tipo de descuento (%)  Inversión Inicial (USD)  
0         3.036898e+07                  11.21               5848413.64  
1         2.297410e+07                  11.21               5848413.64  
2         8.854476e+07                  11.21               5848413.64  
3         1.265774e+08              

In [57]:
datos_sinteticos.head(50)

,Empresa,Año,Inversión Inicial (USD),Tipo de descuento (%),Flujo de Caja (USD),Ingresos Anuales (USD),Costos Anuales (USD),Competidores Actuales,Proyección Demanda (Unidades)
0,Creality,2019,5848413.64,11.21,3.036898e+07,1.260214e+08,9.565245e+07,10,1229
1,Creality,2019,5848413.64,11.21,2.297410e+07,1.260816e+08,1.031075e+08,6,1453
2,Creality,2019,5848413.64,11.21,8.854476e+07,3.280041e+08,2.394594e+08,15,995
3,Creality,2019,5848413.64,11.21,1.265774e+08,4.753638e+08,3.487863e+08,6,1522
4,Creality,2020,5848413.64,11.21,4.940093e+07,4.642285e+08,4.148275e+08,10,1514
5,Creality,2020,5848413.64,11.21,9.083687e+07,5.911545e+08,5.003177e+08,5,1862
6,Creality,2020,5848413.64,11.21,2.683418e+07,1.986344e+08,1.718002e+08,6,752
7,Creality,2020,5848413.64,11.21,5.911594e+07,2.760352e+08,2.169193e+08,8,801
8,Creality,2021,5848413.64,11.21,1.339799e+08,4.771073e+08,3.431273e+08,5,1360
9,Creality,2021,5848413.64,11.21,5.081323e+07,4.941611e+08,4.433479e+08,9,1559


In [58]:
datos_sinteticos.to_excel('sinteticos.xlsx')